In [64]:
import numpy
import urllib
import scipy.optimize
import random
from sklearn import linear_model
import gzip
from collections import defaultdict

In [65]:
import warnings
warnings.filterwarnings("ignore")

In [66]:
def assertFloat(x):
    assert type(float(x)) == float

def assertFloatList(items, N):
    assert len(items) == N
    assert [type(float(x)) for x in items] == [float]*N

In [67]:
f = open("5year.arff", 'r')

In [68]:
# Read and parse the data
while not '@data' in f.readline():
    pass

dataset = []
for l in f:
    if '?' in l: # Missing entry
        continue
    l = l.split(',')
    values = [1] + [float(x) for x in l]
    values[-1] = values[-1] > 0 # Convert to bool
    dataset.append(values)

In [69]:
X = [d[:-1] for d in dataset]
y = [d[-1] for d in dataset]

print(len(X))
print(len(y))

3031
3031


In [70]:
answers = {} # Your answers

In [71]:
def accuracy(pred, y):
    TP_ = numpy.logical_and(pred, y)
    FP_ = numpy.logical_and(pred, numpy.logical_not(y))
    TN_ = numpy.logical_and(numpy.logical_not(pred), numpy.logical_not(y))
    FN_ = numpy.logical_and(numpy.logical_not(pred), y)

    TP = sum(TP_)
    FP = sum(FP_)
    TN = sum(TN_)
    FN = sum(FN_)

    acc = (TP + TN)/len(pred)
    return acc

In [72]:
def BER(pred, y):
    """Evaluate Balanced Error rate of prediction and true values"""
    TP_ = numpy.logical_and(pred, y)
    FP_ = numpy.logical_and(pred, numpy.logical_not(y))
    TN_ = numpy.logical_and(numpy.logical_not(pred), numpy.logical_not(y))
    FN_ = numpy.logical_and(numpy.logical_not(pred), y)

    TP = sum(TP_)
    FP = sum(FP_)
    TN = sum(TN_)
    FN = sum(FN_)

    TPR = TP / (TP + FN)
    TNR = TN / (TN + FP)

    BER = 1 - (0.5*(TPR + TNR))
    return BER


In [73]:
### Question 1

In [74]:
mod = linear_model.LogisticRegression(C=1)
mod.fit(X,y)

pred = mod.predict(X)

In [75]:
acc1 = accuracy(pred, y)
ber1 = BER(pred, y)


In [76]:
answers['Q1'] = [acc1, ber1] # Accuracy and balanced error rate

In [77]:
assertFloatList(answers['Q1'], 2)

In [78]:
### Question 2

In [79]:
mod = linear_model.LogisticRegression(C=1, class_weight='balanced')
mod.fit(X,y)

pred = mod.predict(X)

In [80]:
acc2 = accuracy(pred, y)
ber2 = BER(pred, y)

In [81]:
answers['Q2'] = [acc2, ber2]

In [82]:
assertFloatList(answers['Q2'], 2)

In [83]:
### Question 3

In [84]:
random.seed(3)
random.shuffle(dataset)

In [85]:
X = [d[:-1] for d in dataset]
y = [d[-1] for d in dataset]

In [86]:
Xtrain, Xvalid, Xtest = X[:len(X)//2], X[len(X)//2:(3*len(X))//4], X[(3*len(X))//4:]
ytrain, yvalid, ytest = y[:len(X)//2], y[len(X)//2:(3*len(X))//4], y[(3*len(X))//4:]

In [87]:
len(Xtrain), len(Xvalid), len(Xtest)

(1515, 758, 758)

In [88]:
model = linear_model.LogisticRegression(C=1, class_weight='balanced')
model.fit(Xtrain, ytrain)

LogisticRegression(C=1, class_weight='balanced')

In [89]:
pred_train = model.predict(Xtrain)
pred_valid = model.predict(Xvalid)
pred_test = model.predict(Xtest)

In [90]:
berTrain = BER(pred_train, ytrain)
berValid = BER(pred_valid, yvalid)
berTest = BER(pred_test, ytest)

In [91]:
answers['Q3'] = [berTrain, berValid, berTest]

In [92]:
assertFloatList(answers['Q3'], 3)

In [93]:
### Question 4

In [94]:
def make_pred(X, y, c, data_set):
    """Make a prediction using different regularization coefficients"""
    model = linear_model.LogisticRegression(C=c, class_weight='balanced')
    model.fit(X, y)

    pred = model.predict(data_set)
    return pred

In [95]:
pred_n10000 = make_pred(Xtrain, ytrain, c=10**-4, data_set=Xvalid)
pred_n1000 = make_pred(Xtrain, ytrain, c=10**-3, data_set=Xvalid)
pred_n100 = make_pred(Xtrain, ytrain, c=10**-2, data_set=Xvalid)
pred_n10 = make_pred(Xtrain, ytrain, c=10**-1, data_set=Xvalid)
pred_1 = make_pred(Xtrain, ytrain, c=1, data_set=Xvalid)
pred_10 = make_pred(Xtrain, ytrain, c=10, data_set=Xvalid)
pred_100 = make_pred(Xtrain, ytrain, c=10**2, data_set=Xvalid)
pred_1000 = make_pred(Xtrain, ytrain, c=10**3, data_set=Xvalid)
pred_10000 = make_pred(Xtrain, ytrain, c=10**4, data_set=Xvalid)


In [96]:
BER_n10000 = BER(pred_n10000, yvalid)
BER_n1000 = BER(pred_n1000, yvalid)
BER_n100 = BER(pred_n100, yvalid)
BER_n10 = BER(pred_n10, yvalid)
BER_1 = BER(pred_1, yvalid)
BER_10 = BER(pred_10, yvalid)
BER_100 = BER(pred_100, yvalid)
BER_1000 = BER(pred_1000, yvalid)
BER_10000 = BER(pred_10000, yvalid)

berList = [BER_n10000, BER_n1000, BER_n100,
           BER_n10, BER_1, BER_10, BER_100, BER_1000, BER_10000]


In [97]:
answers['Q4'] = berList

In [98]:
assertFloatList(answers['Q4'], 9)

In [99]:
### Question 5

In [100]:
# check for lowest BER from the regularization pipeline 
print(berList)
print(min(berList))

[0.3288104929895974, 0.31931252826775225, 0.3301673450927183, 0.3179556761646314, 0.3159203980099503, 0.3111714156490276, 0.2955030044582283, 0.29618143050978873, 0.29618143050978873]
0.2955030044582283


In [101]:
c_s = [10**-4, 10**-3, 10**-2, 10**-1, 1, 10, 10**2, 10**3, 10**4] # list of regularization coefficients
best_c_ind= berList.index(min(berList))
bestC = c_s[best_c_ind] 

print(f"Best C at index: {best_c_ind}")
print(f"Best C: {bestC}")

Best C at index: 6
Best C: 100


In [102]:
best_pred = make_pred(Xtrain, ytrain, bestC, Xtest) # create model with the best C value(smallest BER) and test data
ber5 = BER(best_pred, ytest)

In [103]:
answers['Q5'] = [bestC, ber5]

In [104]:
assertFloatList(answers['Q5'], 2)

In [105]:
### Question 6

In [106]:
f = gzip.open("young_adult_10000.json.gz")
dataset = []
for l in f:
    dataset.append(eval(l))

In [107]:
dataTrain = dataset[:9000]
dataTest = dataset[9000:]

print(dataTrain[0])

{'user_id': '8842281e1d1347389f2ab93d60773d4d', 'book_id': '2767052', 'review_id': '248c011811e945eca861b5c31a549291', 'rating': 5, 'review_text': "I cracked and finally picked this up. Very enjoyable quick read - couldn't put it down - it was like crack. \n I'm a bit bothered by the lack of backstory of how Panem and the Hunger Games come about. It is just kind of explained away in a few paragraphs and we are left to accept this very strange world where teenagers are pitted into an arena each year to kill each other? I was expecting it because I've seen Battle Royale, but I would have appreciated knowing more of the backstory of how the world could have come into such a odd state. \n I suppose what makes a book like this interesting is thinking about the strategy of it all. The players are going to be statistically encouraged to band together because they will last longer that way, but by definition of course any partnership will be broken, and the drama of how that unfolds is always 

In [108]:
# Some data structures you might want

usersPerItem = defaultdict(set) # Maps an item to the users who rated it
itemsPerUser = defaultdict(set) # Maps a user to the items that they rated
reviewsPerUser = defaultdict(list)
reviewsPerItem = defaultdict(list)
ratingDict = {} # To retrieve a rating for a specific user/item pair

for d in dataTrain:
    user = d['user_id']
    item = d['book_id']
    review = d['review_id']
    rating = d['rating']
    usersPerItem[item].add(user) # for each item, there is a set of users that gave a rating
    itemsPerUser[user].add(item) # for each user, there is a set of items they've rated
    reviewsPerUser[user].append(d) # for each user, there is a list of reviews the user made
    reviewsPerItem[item].append(d) # for each item, there is a list of reviews that was made for item
    ratingDict[(user, item)] = rating

In [109]:
def Jaccard(s1, s2):
    numerator = len(s1.intersection(s2))
    denominator = len(s1.union(s2))
    return numerator/denominator

In [110]:
def mostSimilar(i, N=10):
    similarities = []        # list of (Jaccard_similarity, item) pair
    users =  usersPerItem[i] # list of users that rated item i 
    for i2 in usersPerItem:
        if i2 == i: continue
        sim = Jaccard(users, usersPerItem[i2]) # compare item that we're looking at to other items, to see if same user have rated it 
        similarities.append((sim, i2))
    similarities.sort(reverse=True)
    
    return similarities[:N]

In [111]:
answers['Q6'] = mostSimilar('2767052', 10)

In [112]:
assert len(answers['Q6']) == 10
assertFloatList([x[0] for x in answers['Q6']], 10)

In [113]:
### Question 7

In [114]:
# def avg_user_rating(user):
#     ratings = []
#     for rev in reviewsPerUser[user]:
#         ratings.append(rev['rating'])
#     if len(ratings) > 0:
#         avg_rating = sum(ratings)/len(ratings)
#         return avg_rating
#     return 0

def avg_item_rating(item):
    """Get the average rating of an item"""
    ratings = []
    for rev in reviewsPerItem[item]:
        ratings.append(rev['rating'])

    if len(ratings) > 0:
        avg_rating = sum(ratings)/len(ratings)
        return avg_rating
    return 0

In [115]:
def predRating(user, item):
    ratings = []
    similarities = []
    for d in  reviewsPerUser[user]:
        i2 = d['book_id']
        if item == i2: continue
        ratings.append(d['rating'] - (avg_item_rating(i2)))                # subtract by the average rating for regularization purposes
        similarities.append(Jaccard(usersPerItem[item], usersPerItem[i2])) # Use Jaccard similaritiy to find similarity between items
    if sum(similarities) > 0:
        weightedRatings = [(x*y) for x,y in zip(ratings, similarities)]
        return avg_item_rating(item) + sum(weightedRatings) / sum(similarities)     # uses the form of function presented in Q7
    else:
        return sum([d['rating'] for d in dataset]) / len(dataset)

        

In [116]:
sse = 0
for d in dataTest:
    sse += (d['rating']-predRating(d['user_id'], d['book_id']))**2

In [ ]:
mse7 = sse/len(dataTest)

In [ ]:
answers['Q7'] = mse7

In [ ]:
assertFloat(answers['Q7'])

In [ ]:
### Question 8

In [ ]:
def predRatingByUser(user, item):
    ratings = []
    similarities = []
    for d in  reviewsPerUser[user]:
        i2 = d['book_id']
        u2 = d['user_id']
        if user == u2: continue
        ratings.append(d['rating'] - (avg_item_rating(i2)))
        similarities.append(Jaccard(itemsPerUser[user], itemsPerUser[u2]))      # USe Jaccard to get similarities between set of items that user rated compared to u2
    if sum(similarities) > 0:
        weightedRatings = [(x*y) for x,y in zip(ratings, similarities)]
        return avg_item_rating(item) + sum(weightedRatings) / sum(similarities)
    else:
        return sum([d['rating'] for d in dataset]) / len(dataset)

In [ ]:
sse = 0
for d in dataTest:
    sse += (d['rating']-predRatingByUser(d['user_id'], d['book_id']))**2

In [ ]:
mse8 = sse/len(dataTest)

In [ ]:
answers['Q8'] = mse8

In [ ]:
assertFloat(answers['Q8'])

In [ ]:
f = open("answers_hw2.txt", 'w')
f.write(str(answers) + '\n')
f.close()